In [2]:
from astropy.io import ascii, fits
from astropy.coordinates import SkyCoord
from astropy.table import Table, Column, MaskedColumn
from astropy.table import QTable
import astropy.units as u
from astropy.constants import c as c

import json

import os
from glob import glob

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
with open('/Users/kaelyn/Documents/Rochester REU/Kaelyn Figures/FOR CODE/SNEcatalog_6.3.19.json', encoding='utf8', errors='ignore') as json_file:
    data = json.load(json_file)

In [4]:
catalog_table = Table()
JSON_keys = data['10Z2-A'].keys()
n = len(data)

catalog_table['ID'] = [i for i in range(n)]

for col_name in JSON_keys:
    catalog_table[col_name] = [None] * n
    
catalog_table['lumdist_kind']  = [None] * n
catalog_table['redshift_kind'] = [None] * n
catalog_table['velocity_kind'] = [None] * n

In [5]:
for i, objname in enumerate(data.keys()):
    record = data[objname]
    
    if len(record['alias']) > 0:
        catalog_table['alias'][i] = ', '.join([h['value'] for h in record['alias']])
        
    if len(record['catalog']) > 0:
        catalog_table['catalog'][i] = record['catalog'][0]
    
    if len(record['claimedtype']) > 0:
        catalog_table['claimedtype'][i] = ', '.join([h['value'] for h in record['claimedtype']])
    
    if len(record['dec']) > 0 and len(record['ra']) > 0:
        coord = SkyCoord(record['ra'][0]['value'], record['dec'][0]['value'], frame ='icrs', unit=('hour','deg'))
        catalog_table['dec'][i] = coord.dec.degree
        catalog_table['ra'][i] = coord.ra.degree

    if len(record['discoverdate']) > 0:
        catalog_table['discoverdate'][i] = ', '.join([h['value'] for h in record['discoverdate']])
        
    if len(record['discoverer']) > 0:
        catalog_table['discoverer'][i] = ', '.join([h['value'] for h in record['discoverer']])
        
    if len(record['download']) > 0:
        catalog_table['download'][i] = record['download'][0]
    
    if len(record['ebv']) > 0:
        catalog_table['ebv'][i] = ', '.join([h['value'] for h in record['ebv']])
    
    if len(record['host']) > 0:
        catalog_table['host'][i] = ', '.join([h['value'] for h in record['host']])

    # Catch "dirty" values that cause an IllegalHourAngle exception.
    try:
        if len(record['hostdec']) > 0 and len(record['hostra']) > 0: 
            h_coord = SkyCoord(record['hostra'][0]['value'], record['hostdec'][0]['value'], frame ='icrs', unit=('hour','deg'))
            catalog_table['hostdec'][i] = h_coord.dec.degree
            catalog_table['hostra'][i] = h_coord.ra.degree
    except:
        catalog_table['hostra'][i] = -999.
        catalog_table['hostdec'][i] = -999.

    if len(record['hostoffsetang']) > 0:
        catalog_table['hostoffsetang'][i] = ', '.join([h['value'] for h in record['hostoffsetang']])
    
    if len(record['hostoffsetdist']) > 0:
        catalog_table['hostoffsetdist'][i] = ', '.join([h['value'] for h in record['hostoffsetdist']])
    
    if len(record['instruments']) > 0:
        catalog_table['instruments'][i] = ', '.join(record['instruments'])
    
    if len(record['lumdist']) > 0:
        lum_ = record['lumdist'][0]
        catalog_table['lumdist'][i] = ', '.join([h['value'] for h in record['lumdist']])
        if 'kind' in lum_:
            lum_kind_ = lum_['kind']
            catalog_table['lumdist_kind'][i] = lum_kind_

    if len(record['maxabsmag']) > 0:
        catalog_table['maxabsmag'][i] = ', '.join([h['value'] for h in record['maxabsmag']])
        
    if len(record['maxappmag']) > 0:
        catalog_table['maxappmag'][i] = ', '.join([h['value'] for h in record['maxappmag']])
        
    if len(record['maxdate']) > 0:
        catalog_table['maxdate'][i] = ', '.join([h['value'] for h in record['maxdate']])
    
    if len(record['name']) > 0:
        catalog_table['name'][i] = record['name'][0]
    
    if len(record['photolink']) > 0:
        catalog_table['photolink'][i] = record['photolink'][0]
        
    if len(record['radiolink']) > 0:
        catalog_table['radiolink'][i] = record['radiolink'][0]
        
    if len(record['redshift']) > 0:
        z_ = record['redshift'][0]
        catalog_table['redshift'][i] = float(record['redshift'][0]['value']) # saving only the first value here
        if 'kind' in z_:
            if isinstance(z_['kind'], list):
                catalog_table['redshift_kind'][i] = ', '.join([h for h in z_['kind']])
            else:
                catalog_table['redshift_kind'][i] = z_['kind']

    if len(record['references']) > 0:
        catalog_table['references'][i] = record['references'][0]

    if len(record['sources']) > 0:
        catalog_table['sources'][i] = record['sources'][0]['name'], record['sources'][0]['url'], record['sources'][0]['alias']

    if len(record['spectralink']) > 0:
        catalog_table['spectralink'][i] = record['spectralink'][0]

    if len(record['velocity']) > 0:
        vel_ = record['velocity'][0]
        catalog_table['velocity'][i] = ', '.join([h['value'] for h in record['velocity']])
        if 'kind' in vel_:
            vel_kind_ = vel_['kind']
            catalog_table['velocity_kind'][i] = vel_kind_

    if len(record['xraylink']) > 0:
        catalog_table['xraylink'][i] = record['xraylink'][0]

In [6]:
# for i, objname in enumerate(data.keys()):
#     record = data[objname]
    
#     if len(record['photolink']) > 0:
#         print(record['photolink'])

In [7]:
catalog_table

ID,alias,catalog,claimedtype,dec,discoverdate,discoverer,download,ebv,host,hostdec,hostoffsetang,hostoffsetdist,hostra,instruments,lumdist,maxabsmag,maxappmag,maxdate,name,photolink,ra,radiolink,redshift,references,sources,spectralink,velocity,xraylink,lumdist_kind,redshift_kind,velocity_kind
int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
0,10Z2-A,sne,None,0.011027777777777779,None,None,None,0.0337,None,None,None,None,None,None,None,None,None,None,10Z2-A,None,161.77716666666666,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
1,10Z2-B,sne,None,0.0077777777777777776,None,None,None,0.0383,None,None,None,None,None,None,None,None,None,None,10Z2-B,None,161.42816666666667,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
2,10Z2-D,sne,None,-0.23130555555555557,None,None,None,0.0318,None,None,None,None,None,None,None,None,None,None,10Z2-D,None,161.92058333333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
3,"1156-62, G296.8-00.3, MWSNR 296.8-00.3",sne,None,-62.583333333333336,None,None,None,None,Milky Way,None,None,None,None,None,None,None,None,None,1156-62,None,179.62499999999997,None,None,2014BASI...42...47G,None,None,None,None,None,None,None
4,13Z3-A,sne,None,-0.23947222222222223,None,None,None,0.0241,None,None,None,None,None,None,None,None,None,None,13Z3-A,None,205.87024999999997,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
5,13Z3-C,sne,None,-0.30294444444444446,None,None,None,0.0228,None,None,None,None,None,None,None,None,None,None,13Z3-C,None,206.3599583333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
6,13Z3-D,sne,None,0.11247222222222222,None,None,None,0.0232,None,None,None,None,None,None,None,None,None,None,13Z3-D,None,205.9267083333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
7,13Z3-H,sne,None,0.13444444444444445,None,None,None,0.0238,None,None,None,None,None,None,None,None,None,None,13Z3-H,None,206.36229166666664,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
8,13Z3-I,sne,None,-0.34875,None,None,None,0.022,None,None,None,None,None,None,None,None,None,None,13Z3-I,None,206.07704166666664,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None


In [8]:
### Checking to see if any elements in the table are lists.
# for col_name in catalog_table.colnames:
#     for i in range(n):
#         if isinstance(catalog_table[col_name][i], list):
#             print(col_name, i, catalog_table[col_name][i])

In [9]:
### Checking for encoding errors, reason being astropy is using ascii encoder, 
### while JSON was read in using utf8 which allows more characters than ascii.
# for col_name in ['alias', 'discoverer', 'name']:
#     for i in range(n):
#         try:
#             catalog_table[col_name][i].encode('ascii')
#         except:
#             if catalog_table[col_name][i] is not None:
#                 print(i, col_name, catalog_table[col_name][i])

In [10]:
# catalog_table['discoverer'][36260] = 'K. Sarneczky et al.'
# catalog_table['discoverer'][51575] = 'Sarneczky, Kuli, K. Sarneczky et al.'
# catalog_table['discoverer'][51672] = 'Kuli, Sarneczky, Z. Kuli, K. Sa'
# catalog_table['discoverer'][51697] = 'Sarneczky, Kuli, K. Sarneczky et al.'
# catalog_table['discoverer'][52003] = 'Sarneczky, Vinko, Wheeler, K. Sarneczky et al.'
# catalog_table['discoverer'][53215] = 'iPTF, K. Sarneczky et al.'
# catalog_table['discoverer'][53471] = 'Mederic Boquien, Dalya Baron, Lluis Galbany, Dovi Poznanski'
# catalog_table['alias'][57699] = 'SSS120810:231802-560926, SSS120814:221352-124146'
# catalog_table['name'][57699] = 'SSS120810:231802-560926'

In [11]:
print(catalog_table['references'][0])
type(catalog_table['references'][0])

2005A&A...430...83C,2011ApJ...737..103S


str

In [12]:
obj481_table = Table.read('/Users/kaelyn/Documents/Rochester REU/Kaelyn Figures/FOR CODE/petroMag_rBand_rabsmag.txt', format='ascii.commented_header')

In [13]:
obj481_table

ID,redshift,petroMag_r,rabsmag
int64,float64,float64,float64
1584,0.08,17.40083,-19.631426768273315
465,0.054,16.76493,-19.423829364519463
419,0.06227,16.42431,-20.05545917782679
48713,0.01697,13.93589,-19.773378136188466
736,0.073,17.55096,-19.258872944899753
1456,0.094,15.97771,-21.385874150488046
55449,0.035322,16.50294,-18.745529589593506
4568,0.05,16.91579,-19.07635869297333
52219,0.03013,16.05543,-18.86813363066817


In [14]:
# Append references to final objects

obj481_table['references'] = [None]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = obj481_table['ID'] == SN_ID
    obj481_table['references'][idx_bool] = catalog_table['references'][i]

In [15]:
obj481_table

ID,redshift,petroMag_r,rabsmag,references
int64,float64,float64,float64,object
1584,0.08,17.40083,-19.631426768273315,"2016A&A...594A..13P,2011ApJ...737..103S"
465,0.054,16.76493,-19.423829364519463,"2016A&A...594A..13P,2011ApJ...737..103S"
419,0.06227,16.42431,-20.05545917782679,2016A&A...594A..13P
48713,0.01697,13.93589,-19.773378136188466,"2016AJ....151...33G,2017ApJ...850...89G,2008ApJ...673..999P,2012A&A...538A.120L,2014ApJ...786...67A"
736,0.073,17.55096,-19.258872944899753,"2016A&A...594A..13P,2011ApJ...737..103S"
1456,0.094,15.97771,-21.385874150488046,2016A&A...594A..13P
55449,0.035322,16.50294,-18.745529589593506,2016A&A...594A..13P
4568,0.05,16.91579,-19.07635869297333,2016A&A...594A..13P
52219,0.03013,16.05543,-18.86813363066817,"2013CBET.3424....1G,2016A&A...594A..13P,2008yCat....1.2024B,2011ApJ...737..103S"


In [16]:
ref_table = obj481_table['ID', 'references']

In [17]:
# ref_table.show_in_notebook()

In [18]:
ref = np.unique(obj481_table['references'])
print(ref)

                                             references                                            
---------------------------------------------------------------------------------------------------
1961PASP...73..185Z,2012A&A...538A.120L,2016A&A...594A..13P,1993PASP..105.1250P,1990ApJ...350..110C
1967AN....289..247R,2012A&A...538A.120L,2016A&A...594A..13P,1993PASP..105.1250P,2011ApJ...737..103S
1973MmSAI..44...65B,1989nane.conf..284C,2012A&A...538A.120L,1970IAUC.2287....1W,2016A&A...594A..13P
1978A&AS...34..387C,2012A&A...538A.120L,1975IAUC.2878....4H,2016A&A...594A..13P,1993BICDS..42...17T
1978A&AS...34..387C,2012A&A...538A.120L,2016A&A...594A..13P,1977IAUC.3037....3G,1993BICDS..42...17T
1978ZhPhy..90..565K,2012A&A...538A.120L,1973IAUC.2521....1V,2016A&A...594A..13P,1990ApJ...350..110C
1989IAUC.4903....3S,2012A&A...538A.120L,2016A&A...594A..13P,1993BICDS..42...17T,2014MNRAS.444.2428H
1989IAUC.6122....1K,2012A&A...538A.120L,2008ApJ...673..999P,1994IAUC.6122....3N,2016A&A...594A..13P


In [19]:
np.ma.count(ref)

395

In [20]:
u, indices = np.unique(obj481_table['references'], return_index=True)

In [21]:
u_ref_table = Table()
u_ref_table['references'] = u

In [22]:
u_ref_table['ID'] = -999

for i in range(len(obj481_table)):
    SN_ID = obj481_table['references'][i]
    idx_bool = u_ref_table['references'] == SN_ID
    u_ref_table['ID'][idx_bool] = obj481_table['ID'][i]

In [23]:
u_ref_table = u_ref_table['ID','references']

In [24]:
u_ref_table

ID,references
int64,object
46292,"1961PASP...73..185Z,2012A&A...538A.120L,2016A&A...594A..13P,1993PASP..105.1250P,1990ApJ...350..110C"
46381,"1967AN....289..247R,2012A&A...538A.120L,2016A&A...594A..13P,1993PASP..105.1250P,2011ApJ...737..103S"
46465,"1973MmSAI..44...65B,1989nane.conf..284C,2012A&A...538A.120L,1970IAUC.2287....1W,2016A&A...594A..13P"
46558,"1978A&AS...34..387C,2012A&A...538A.120L,1975IAUC.2878....4H,2016A&A...594A..13P,1993BICDS..42...17T"
46587,"1978A&AS...34..387C,2012A&A...538A.120L,2016A&A...594A..13P,1977IAUC.3037....3G,1993BICDS..42...17T"
46517,"1978ZhPhy..90..565K,2012A&A...538A.120L,1973IAUC.2521....1V,2016A&A...594A..13P,1990ApJ...350..110C"
46793,"1989IAUC.4903....3S,2012A&A...538A.120L,2016A&A...594A..13P,1993BICDS..42...17T,2014MNRAS.444.2428H"
47073,"1989IAUC.6122....1K,2012A&A...538A.120L,2008ApJ...673..999P,1994IAUC.6122....3N,2016A&A...594A..13P"
46753,"1989SvA....15..301Y,2012A&A...538A.120L,1990A&A...228...61C,1987IAUC.4374....3A,2016A&A...594A..13P"


In [25]:
# u_ref_table.write('ref_obj481.txt', format= 'ascii.commented_header', overwrite=True)

In [26]:
# u_ref_table.show_in_notebook()

In [27]:
catalog_table

ID,alias,catalog,claimedtype,dec,discoverdate,discoverer,download,ebv,host,hostdec,hostoffsetang,hostoffsetdist,hostra,instruments,lumdist,maxabsmag,maxappmag,maxdate,name,photolink,ra,radiolink,redshift,references,sources,spectralink,velocity,xraylink,lumdist_kind,redshift_kind,velocity_kind
int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
0,10Z2-A,sne,None,0.011027777777777779,None,None,None,0.0337,None,None,None,None,None,None,None,None,None,None,10Z2-A,None,161.77716666666666,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
1,10Z2-B,sne,None,0.0077777777777777776,None,None,None,0.0383,None,None,None,None,None,None,None,None,None,None,10Z2-B,None,161.42816666666667,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
2,10Z2-D,sne,None,-0.23130555555555557,None,None,None,0.0318,None,None,None,None,None,None,None,None,None,None,10Z2-D,None,161.92058333333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
3,"1156-62, G296.8-00.3, MWSNR 296.8-00.3",sne,None,-62.583333333333336,None,None,None,None,Milky Way,None,None,None,None,None,None,None,None,None,1156-62,None,179.62499999999997,None,None,2014BASI...42...47G,None,None,None,None,None,None,None
4,13Z3-A,sne,None,-0.23947222222222223,None,None,None,0.0241,None,None,None,None,None,None,None,None,None,None,13Z3-A,None,205.87024999999997,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
5,13Z3-C,sne,None,-0.30294444444444446,None,None,None,0.0228,None,None,None,None,None,None,None,None,None,None,13Z3-C,None,206.3599583333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
6,13Z3-D,sne,None,0.11247222222222222,None,None,None,0.0232,None,None,None,None,None,None,None,None,None,None,13Z3-D,None,205.9267083333333,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
7,13Z3-H,sne,None,0.13444444444444445,None,None,None,0.0238,None,None,None,None,None,None,None,None,None,None,13Z3-H,None,206.36229166666664,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None
8,13Z3-I,sne,None,-0.34875,None,None,None,0.022,None,None,None,None,None,None,None,None,None,None,13Z3-I,None,206.07704166666664,None,None,"2005A&A...430...83C,2011ApJ...737..103S",None,None,None,None,None,None,None


In [28]:
all_481 = Table()
all_481['ID'] = obj481_table['ID']

all_481['alias'] = [None]
all_481['catalog'] = [None]
all_481['claimedtype'] = [None]
all_481['dec'] = [None]
all_481['discoverdate'] = [None]
all_481['discoverer'] = [None]
all_481['download'] = [None]
all_481['ebv'] = [None]
all_481['host'] = [None]
all_481['hostdec'] = [None]
all_481['hostoffsetang'] = [None]
all_481['hostoffsetdist'] = [None]
all_481['hostra'] = [None]
all_481['instruments'] = [None]
all_481['lumdist'] = [None]
all_481['maxabsmag'] = [None]
all_481['maxappmag'] = [None]
all_481['maxdate'] = [None]
all_481['name'] = [None]
all_481['photolink'] = [None]
all_481['ra'] = [None]
all_481['radiolink'] = [None]
all_481['redshift'] = [None]
all_481['references'] = [None]
all_481['sources'] = [None]
all_481['spectralink'] = [None]
all_481['velocity'] = [None]
all_481['xraylink'] = [None]
all_481['lumdist_kind'] = [None]
all_481['redshift_kind'] = [None]
all_481['velocity_kind'] = [None]

In [29]:
for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['alias'][idx_bool] = catalog_table['alias'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['catalog'][idx_bool] = catalog_table['catalog'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['claimedtype'][idx_bool] = catalog_table['claimedtype'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['dec'][idx_bool] = catalog_table['dec'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['discoverdate'][idx_bool] = catalog_table['discoverdate'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['discoverer'][idx_bool] = catalog_table['discoverer'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['download'][idx_bool] = catalog_table['download'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['ebv'][idx_bool] = catalog_table['ebv'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['host'][idx_bool] = catalog_table['host'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['hostdec'][idx_bool] = catalog_table['hostdec'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['hostoffsetang'][idx_bool] = catalog_table['hostoffsetang'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['hostoffsetdist'][idx_bool] = catalog_table['hostoffsetdist'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['hostra'][idx_bool] = catalog_table['hostra'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['instruments'][idx_bool] = catalog_table['instruments'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['lumdist'][idx_bool] = catalog_table['lumdist'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['maxabsmag'][idx_bool] = catalog_table['maxabsmag'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['maxappmag'][idx_bool] = catalog_table['maxappmag'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['maxappmag'][idx_bool] = catalog_table['maxappmag'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['maxdate'][idx_bool] = catalog_table['maxdate'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['name'][idx_bool] = catalog_table['name'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['photolink'][idx_bool] = catalog_table['photolink'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['ra'][idx_bool] = catalog_table['ra'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['radiolink'][idx_bool] = catalog_table['radiolink'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['redshift'][idx_bool] = catalog_table['redshift'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['references'][idx_bool] = catalog_table['references'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['spectralink'][idx_bool] = catalog_table['spectralink'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['radiolink'][idx_bool] = catalog_table['radiolink'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['velocity'][idx_bool] = catalog_table['velocity'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['xraylink'][idx_bool] = catalog_table['xraylink'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['lumdist_kind'][idx_bool] = catalog_table['lumdist_kind'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['redshift_kind'][idx_bool] = catalog_table['redshift_kind'][i]

for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    all_481['velocity_kind'][idx_bool] = catalog_table['velocity_kind'][i]

In [30]:
for i in range(len(catalog_table)):
    SN_ID = catalog_table['ID'][i]
    idx_bool = all_481['ID'] == SN_ID
    print(catalog_table['sources'][i])
    all_481['sources'][idx_bool] = catalog_table['sources'][i]

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


ValueError: NumPy boolean array indexing assignment cannot assign 3 input values to the 0 output values where the mask is true

In [31]:
all_481

ID,alias,catalog,claimedtype,dec,discoverdate,discoverer,download,ebv,host,hostdec,hostoffsetang,hostoffsetdist,hostra,instruments,lumdist,maxabsmag,maxappmag,maxdate,name,photolink,ra,radiolink,redshift,references,sources,spectralink,velocity,xraylink,lumdist_kind,redshift_kind,velocity_kind
int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
1584,"AT2018bdz, C17-0110",sne,Candidate,-1.2396805555555557,2018/04/13,KEGS,None,0.0236,SDSS J133733.14-011422.9,-1.2396944444444444,1.321,1.779,204.3880833333333,DECAM (i),380,-16,21.57,2018/04/13,AT2018bdz,"2,-0.926,0.0428",204.38844999999998,None,0.08,"2016A&A...594A..13P,2011ApJ...737..103S",None,None,20000,None,None,host,None
465,AT2012iq,sne,Candidate,-0.1305777777777778,2012/04/22,Claude Cornen,None,0.0288,SDSS J142211.75-000752.6,-0.13127777777777777,4.694,5.182,215.5489583333333,None,249,-17.7,19.2,2012/04/21,AT2012iq,"1,0",215.55005833333334,None,0.054,"2016A&A...594A..13P,2011ApJ...737..103S",None,None,16000,None,None,host,None
419,AT1999gz,sne,Candidate,0.816625,1999/03/20,Galaxy Zoo,None,None,SDSS J115731.45+004901.6,0.8171111111111111,7.613,9.437,179.38104166666662,None,288.47,-16.835,20.4,1999/03/19,AT1999gz,"1,0",179.38309999999998,None,0.06227,2016A&A...594A..13P,None,None,18090,None,None,host,None
48713,SN2003ej,sne,"II P, II",0.7251111111111112,2003/05/20,LOTOSS,None,0.0158,UGC 7820,0.7175,28.11,10.26,189.79799999999997,"B, V, I",76.076,-17.968,16.42,2003/05/23,SN2003ej,"28,-0.393,67.4",189.79625,None,0.01697,"2016AJ....151...33G,2017ApJ...850...89G,2008ApJ...673..999P,2012A&A...538A.120L,2014ApJ...786...67A",None,"7,-1.47,5.41",5131,None,None,host,host
736,AT2016jie,sne,Candidate,0.6890000000000001,2016/06/07,Galaxy Zoo,None,0.0291,SDSS J131325.47+004114.5,0.6873611111111111,5.934,8.58,198.356125,DECAM (R),341,-18.3,19.26,2016/06/07,AT2016jie,"1,-0.441",198.35629999999998,None,0.073,"2016A&A...594A..13P,2011ApJ...737..103S",None,None,21000,None,None,host,None
1456,AT2017kdr,sne,Candidate,1.112538888888889,2017/03/02,Galaxy Zoo,None,None,SDSS J101249.79+010635.9,1.1099722222222224,11.71,21.28,153.2074583333333,DECAM (z),445,-15.6,22.5,2017/03/02,AT2017kdr,"1,-0.418",153.2094583333333,None,0.094,2016A&A...594A..13P,None,None,27000,None,None,host,None
55449,"SN2018jrj, AT2018jrj, ZTF18acrxpzx, PS19lr",sne,II,64.24737777777779,2018/11/28,"ZTF, Pan-STARRS1, ATLAS, ZTF",None,None,SDSS J111103.62+641445.8,64.24605555555556,5.354,3.887,167.76508333333334,"GPC (i), GPC1 (i), ZTF-Cam (g, r)",160.51,-18.0998,17.89,2018/12/09,SN2018jrj,"7,-10.7,43.5",167.76665,None,0.035322,2016A&A...594A..13P,None,"1,2.83",10402,None,None,None,None
4568,AT2019cjt,sne,Candidate,49.120222222222225,2019/03/25,TNTS,None,None,2MASS J08354255+4907116,49.11988888888889,1.236,1.141,128.92729166666666,None,230,-18,19,2019/03/24,AT2019cjt,"1,0",128.92716666666666,None,0.05,2016A&A...594A..13P,None,None,10000,None,None,host,None
52219,"SN2013ac, <a id=""PSNJ09454879+5840073"">PSN J09454879+5840073</a>",sne,II P,58.66869444444444,2013/02/15,"Zhang et al., THU-NAOC Transient Survey (TNTS)",None,0.0101,PGC 3086846,58.66916666666666,6.392,3.983,146.28333333333333,None,136.4,-17.642,18,2013/02/17,SN2013ac,"1,0",146.286625,None,0.03013,"2013CBET.3424....1G,2016A&A...594A..13P,2008yCat....1.2024B,2011ApJ...737..103S",None,None,8897,None,None,heliocentric,host


In [32]:
# all_481.write('481objects.txt', format= 'ascii.commented_header', overwrite=True)